In [1]:
suppressMessages(library("CARD", quietly = T))
suppressMessages(library("ggplot2", quietly = T))
suppressMessages(library("reshape2", quietly = T))
suppressMessages(library("gtools", quietly = T))
suppressMessages(library("scatterpie", quietly = T))
suppressMessages(library("glue", quietly = T))

Warning message:
“replacing previous import ‘RcppML::nmf’ by ‘NMF::nmf’ when loading ‘CARD’”


In [2]:
# devtools::install_github('xuranw/MuSiC')

In [3]:
# load spatial data
key = "HFD14"
subId = "macrophage_"
# subId = ""
inputDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"

sptPath = glue("{inputDir}{key}_{subId}spt.csv")
spt.df <- read.csv(sptPath)
rownames(spt.df) <- spt.df[,1]
spt.df <- spt.df[,-1]
spt <- as.matrix(spt.df)
dim(spt)

# load spatial coordinates
xyPath = glue("{inputDir}{key}_{subId}coords.csv")
xy.df <- read.csv(xyPath)
rownames(xy.df) <- xy.df[,1]
xy.df <- xy.df[,-1]
dim(xy.df)

# load the rna seq data
rnaPath = glue("{inputDir}{key}_{subId}rna.csv")
rna.df <- read.csv(rnaPath)
rownames(rna.df) <- rna.df[,1]
rna.df <- rna.df[,-1]
rna <- as.matrix(rna.df)
dim(rna)

# load the annotations
metaPath = glue("{inputDir}{key}_{subId}clusters.csv")
meta.df <- read.csv(metaPath)
rownames(meta.df) <- meta.df$cellId
dim(meta.df)

[1] 31053  1994

[1] 1994    2

[1] 31053  6436

[1] 6436    3

In [4]:
CARD_obj = createCARDObject(
	sc_count = rna,
	sc_meta = meta.df,
	spatial_count = spt,
	spatial_location = xy.df,
	ct.varname = "cellType",
	ct.select = unique(meta.df$cellType),
	sample.varname = "sampleInfo",
	minCountGene = 20,
	minCountSpot = 5)

## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...


In [5]:
CARD_obj = suppressMessages(CARD_deconvolution(CARD_object = CARD_obj))

## create reference matrix from scRNASeq...
## Select Informative Genes! ...
## Deconvolution Starts! ...
## Deconvolution Finish! ...


In [6]:
props.df <- as.data.frame(CARD_obj@Proportion_CARD)

outDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDOuputs/"
outpath = glue("{outDir}{key}_{subId}CARD.csv")

write.csv(props.df, 
          outpath, 
          row.names=TRUE)

dim(props.df)

head(props.df)

[1] 1991   11

,M2,M1,Monocytes,T cells,M5,NK cells,Dendritic cells,Stromal cells,B cells,M4,M3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACATTTCCCGGATT.1,0.03642540,0.25133895,0.06975495,0.026716252,0.014474067,0.024719757,0.055228746,0.40783756,0.06265205,0.01884796,0.032004315
AAACCGGGTAGGTACC.1,0.08062658,0.58662310,0.12289673,0.006747509,0.007289972,0.001327306,0.023560875,0.09139413,0.01233849,0.02859301,0.038602290
AAACCGTTCGTCCAGG.1,0.01581330,0.22899183,0.16583649,0.036051874,0.003334284,0.119806615,0.066582466,0.19961083,0.05404725,0.04250378,0.067421276
AAACCTAAGCAGCCGG.1,0.01439056,0.18963856,0.17036426,0.092808081,0.005847792,0.145929785,0.032867037,0.18243907,0.08822633,0.02832671,0.049161825
AAACCTCATGAAGTTG.1,0.06188912,0.26647808,0.10882667,0.034574570,0.044545315,0.104600520,0.040395652,0.16166471,0.06359788,0.05689028,0.056537213
AAACGGGCGTACGGGT.1,0.02541731,0.02487756,0.05831213,0.041947123,0.040203159,0.006926108,0.002005388,0.73596646,0.03726648,0.02211053,0.004967763


In [7]:
# # pieViz <- function(proportion, spatial_location){
# #     res_CARD = as.data.frame(proportion)
# #     res_CARD = res_CARD[,mixedsort(colnames(res_CARD))]
# #     location = as.data.frame(spatial_location) 
# #     colors = colorRampPalette(c("Blue", "Red"))(ncol(res_CARD))
# #     data = cbind(res_CARD,location)
# #     ct.select = colnames(res_CARD)
# #     p = ggplot() + 
# #         geom_scatterpie(aes(x=x, y=y, r = 1), 
# #                         data=data,
# #                         cols=ct.select,color=NA) + 
# #         coord_fixed(ratio = 1) + 
# #         scale_fill_manual(values = colors) 
# #     return(p)
    
# # }

# p <- CARD.visualize.pie(proportion = CARD_obj@Proportion_CARD,
#                         spatial_location = CARD_obj@spatial_location)
                   
# ggsave("cardFigures/mixtures.png",
#        width = 50, 
#        height = 50, 
#        units = "cm")

# print('done')

In [8]:
visCells <- function(proportion, spatial_location, ct.visualize, colors, NumCols, pntSize){
    res_CARD = as.data.frame(proportion)
    res_CARD = res_CARD[,order(colnames(res_CARD))]
    location = as.data.frame(spatial_location)
    if(sum(rownames(res_CARD)==rownames(location))!= nrow(res_CARD)){
    stop("The rownames of proportion data does not match with the rownames of spatial location data")
    }
    ct.select = ct.visualize
    res_CARD = res_CARD[,ct.select]
    res_CARD_scale = as.data.frame(apply(res_CARD,2,function(x){
    (x - min(x)) / (max(x) - min(x))
    } ))
    res_CARD_scale$x = as.numeric(location$x)
    res_CARD_scale$y = as.numeric(location$y)
    mData = melt(res_CARD_scale,id.vars = c("x","y"))
    colnames(mData)[3] <- "Cell_Type"
    b = c(0,1)
    
    p = ggplot(mData, aes(x, y))  +
    geom_point(aes(colour = value), size = pntSize) +
    scale_color_gradientn(colours = colors) + 
    scale_y_reverse() +
    facet_wrap(~Cell_Type,ncol = NumCols)+ 
    coord_fixed()+
    theme(axis.text = element_blank(),
          axis.ticks = element_blank(),
          axis.title = element_blank(),
          panel.background = element_blank(),
          plot.background = element_blank(),
          panel.grid.major = element_blank(), 
          panel.grid.minor = element_blank(),
         )
    return(p)
}

p <- visCells(CARD_obj@Proportion_CARD,
         CARD_obj@spatial_location,
         unique(meta.df$cellType),
         c("lightblue","lightyellow","red"),
         4, 
         0.1)

figDir = "cardFigures/"
outpath = glue("{figDir}{key}_{subId}spots.png")
ggsave(outpath,
       width = 20, 
       height = 20, 
       units = "cm")

print('done')

[1] "done"


In [9]:
CARD_obj = CARD.imputation(CARD_obj,
                           NumGrids = 10000,
                           ineibor = 9,
                           exclude = NULL)

location_imputation = cbind.data.frame(x=as.numeric(sapply(strsplit(rownames(CARD_obj@refined_prop),split="x"),"[",1)),
                                       y=as.numeric(sapply(strsplit(rownames(CARD_obj@refined_prop),split="x"),"[",2)))

rownames(location_imputation) = rownames(CARD_obj@refined_prop)

p <-visCells(
	CARD_obj@refined_prop,                         
	location_imputation,            
	unique(meta.df$cellType),                    
	c("lightblue","lightyellow","red"),    
	NumCols = 4,
    pntSize = 0.1)     

outpath = glue("{figDir}{key}_{subId}imputed.png")
ggsave(outpath,
       width = 20, 
       height = 20, 
       units = "cm")

print('done')

## The rownames of locations are matched ...
## Make grids on new spatial locations ...
[1] "done"


In [10]:
df <- as.data.frame(CARD_obj@refined_prop)
head(df)

head(location_imputation)

,M2,M1,Monocytes,T cells,M5,NK cells,Dendritic cells,Stromal cells,B cells,M4,M3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
331x476,0.04808219,0.2125914,0.1084684,0.04448411,0.03515460,0.08570093,0.07932952,0.1966562,0.06772388,0.05082444,0.07098441
331x487,0.04805414,0.2126062,0.1084928,0.04448327,0.03500898,0.08576047,0.07938642,0.1966851,0.06770905,0.05080409,0.07100952
320x498,0.04806115,0.2126148,0.1084869,0.04448304,0.03503581,0.08574718,0.07937389,0.1966730,0.06771177,0.05080774,0.07100476
331x498,0.04802734,0.2128199,0.1085192,0.04447537,0.03471464,0.08584306,0.07946831,0.1966340,0.06767883,0.05076130,0.07105801
342x498,0.04799812,0.2126186,0.1085416,0.04448215,0.03473121,0.08587720,0.07949777,0.1967497,0.06768071,0.05076546,0.07105752
320x509,0.04803144,0.2128487,0.1085161,0.04447440,0.03471189,0.08583834,0.07946426,0.1966176,0.06767851,0.05076066,0.07105813


,x,y
,<dbl>,<dbl>
331x476,331,476
331x487,331,487
320x498,320,498
331x498,331,498
342x498,342,498
320x509,320,509


In [11]:
# save the high res file
# head(location_imputation)

props.df.refined <- as.data.frame(CARD_obj@refined_prop)

outDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDOuputs/"
outpath = glue("{outDir}{key}_{subId}CARD_refined.csv")

write.csv(props.df, 
          outpath, 
          row.names=TRUE)

dim(props.df.refined)
head(props.df.refined)

[1] 5571   11

,M2,M1,Monocytes,T cells,M5,NK cells,Dendritic cells,Stromal cells,B cells,M4,M3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
331x476,0.04808219,0.2125914,0.1084684,0.04448411,0.03515460,0.08570093,0.07932952,0.1966562,0.06772388,0.05082444,0.07098441
331x487,0.04805414,0.2126062,0.1084928,0.04448327,0.03500898,0.08576047,0.07938642,0.1966851,0.06770905,0.05080409,0.07100952
320x498,0.04806115,0.2126148,0.1084869,0.04448304,0.03503581,0.08574718,0.07937389,0.1966730,0.06771177,0.05080774,0.07100476
331x498,0.04802734,0.2128199,0.1085192,0.04447537,0.03471464,0.08584306,0.07946831,0.1966340,0.06767883,0.05076130,0.07105801
342x498,0.04799812,0.2126186,0.1085416,0.04448215,0.03473121,0.08587720,0.07949777,0.1967497,0.06768071,0.05076546,0.07105752
320x509,0.04803144,0.2128487,0.1085161,0.04447440,0.03471189,0.08583834,0.07946426,0.1966176,0.06767851,0.05076066,0.07105813


In [12]:
# pp <- CARD.visualize.gene(
# 	spatial_expression = CARD_obj@refined_expression,
# 	spatial_location = location_imputation,
# 	gene.visualize = c("LPL", "RPL4"),
# 	colors = NULL,
# 	NumCols = 2)

# ggsave("cardFigures/genes.png",
#        width = 20, 
#        height = 20, 
#        units = "cm")

# print('done')

In [13]:
# exp.df <- as.data.frame(rowSums(spt))
# exp.df$names <- rownames(exp.df)
# exp.df.sorted <- exp.df[order(exp.df$rowSums, decreasing = TRUE),]
# head(exp.df.sorted, 20)

In [14]:
# ## select the cell type that we are interested
# ct.visualize = unique(meta.df$cellType)

# ## visualize the spatial distribution of the cell type proportion
# p <- CARD.visualize.prop(
# 	proportion = CARD_obj@Proportion_CARD,        
# 	spatial_location = CARD_obj@spatial_location, 
# 	ct.visualize = ct.visualize,
#     colors = c("lightblue","lightyellow","red"),
#     NumCols=4
# )

# ggsave("test.png", width = 20, height = 20, units = "cm")



# # print(p2)